# Seleção de variáveis para treinamento de um modelo capaz de prever qual a classificação da proteína

- O modelo treinado irá tentar prever qual a classificação da estrutura molecular
- Os tipos de classificação serão agrupados
- Serão avaliadas features diversas, que envolvem resultados experimentais de uma determinada molécula

In [1]:
from dotenv import load_dotenv
import os
import sqlalchemy
import pandas as pd
import plotly.express as px

from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
load_dotenv()

mysql_credentials = {
    'user': os.getenv("MYSQL_USER"),
    'pwd': os.getenv("MYSQL_PASSWORD"),
    'host': os.getenv("MYSQL_HOST"),
    'port': os.getenv("MYSQL_PORT"),
    'db': os.getenv("MYSQL_DB")
}

In [112]:
user = mysql_credentials['user']
password = mysql_credentials['pwd']
host = mysql_credentials['host']
port = mysql_credentials['port']
database = mysql_credentials['db']

url = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
conn = sqlalchemy.create_engine(url)

In [113]:
query = '''
    select 
        te.CLASSIFICATION_STR as y,
        inf.CRYSTALLIZATIONTEMPK_FLOAT as temperatura_cristalizacao_k,
        inf.RESIDUECOUNT_INT as contagem_residuos,
        inf.RESOLUTION_FLOAT as medida_de_resolucao,
        inf.STRUCTUREMOLECULARWEIGHT_FLOAT as peso_molecular,
        va.PHVALUE_FLOAT as ph,
        va.DENSITYMATTHEWS_FLOAT as densidade_matthews,
        va.DENSITYPERCENTSOL_FLOAT as densidade_percentual_solucao,
        te.EXPERIMENTALTECHNIQUE_STR as tecnica_experimental
    from protein_data_bank.tipo_estrutura te
    inner join protein_data_bank.infos_estrutura inf
        on inf.STRUCTUREID_STR = te.STRUCTUREID_STR
    inner join protein_data_bank.valores_analiticos va
        on va.STRUCTUREID_STR = te.STRUCTUREID_STR
'''

In [114]:
df = pd.read_sql_query(query, conn)

In [115]:
df.head()

,y,temperatura_cristalizacao_k,contagem_residuos,medida_de_resolucao,peso_molecular,ph,densidade_matthews,densidade_percentual_solucao,tecnica_experimental
0,DNA,277.0,24,2.20,7637.17,7.0,2.28,46.06,X-RAY DIFFRACTION
1,DNA,277.0,6,1.90,2337.73,6.6,2.90,57.63,X-RAY DIFFRACTION
2,DNA,277.0,24,2.25,7374.83,6.6,2.29,46.25,X-RAY DIFFRACTION
3,DNA,281.0,24,2.50,7356.81,7.4,2.35,47.59,X-RAY DIFFRACTION
4,DNA,277.0,12,2.55,3663.39,6.5,3.01,59.09,X-RAY DIFFRACTION


#### Dicionário de Dados

- **y** (*str*): O tipo da proteína que queremos prever.

- **temperatura_cristalizacao_k** (*float*): A temperatura na qual a substância se transformou de um estado líquido para um estado sólido cristalino.

- **contagem_residuos** (*int*): O número total de resíduos ou componentes individuais dentro da molécula, no nosso caso são os aminoácidos de uma proteína.

- **medida_de_resolucao** (*float*): Uma medida da clareza ou detalhe com que uma estrutura molecular, como uma proteína, pode ser visualizada através de técnicas como cristalografia.

- **peso_molecular** (*float*): A massa total de uma molécula, determinada pela soma dos pesos atômicos de todos os átomos constituintes.

- **pH** (*float*): Uma medida da acidez ou basicidade de uma solução aquosa. Indica a concentração de íons de hidrogênio presentes.

- **densidade_matthews** (*float*): Uma medida da densidade de moléculas numa célula unitária cristalina, utilizada em estudos de cristalografia.

- **densidade_percent_solucao** (*float*): A concentração de uma solução, expressa como a quantidade de soluto presente em relação ao volume total da solução.

- **tecnica_experimental** (*str*): A técnica experimental utilizada para obter parte dos dados relacionados a estrutura molecular. 

Tratamento nos dados textuais:

In [116]:
def text_treat(text: pd.Series):
    map_clean = ['\n', '\r', '-', ',', '/', ' ']
    for map in map_clean: 
        text = text.str.replace(map, '')
    return text

df['y'] = text_treat(df['y'])
df['tecnica_experimental'] = text_treat(df['tecnica_experimental'])

In [117]:
df_dummy = pd.get_dummies(df['tecnica_experimental'], prefix='tecnica_experimental_', dtype=int)
df = df.drop(columns=['tecnica_experimental'])
df = pd.concat([df, df_dummy], axis=1)

In [118]:
df.head()

,y,temperatura_cristalizacao_k,contagem_residuos,medida_de_resolucao,peso_molecular,ph,densidade_matthews,densidade_percentual_solucao,tecnica_experimental__ELECTRONCRYSTALLOGRAPHY,tecnica_experimental__EPRXRAYDIFFRACTION,tecnica_experimental__NEUTRONDIFFRACTION,tecnica_experimental__NEUTRONDIFFRACTIONXRAYDIFFRACTION,tecnica_experimental__POWDERDIFFRACTION,tecnica_experimental__SOLUTIONSCATTERINGXRAYDIFFRACTION,tecnica_experimental__XRAYDIFFRACTION,tecnica_experimental__XRAYDIFFRACTIONEPR,tecnica_experimental__XRAYDIFFRACTIONNEUTRONDIFFRACTION
0,DNA,277.0,24,2.20,7637.17,7.0,2.28,46.06,0,0,0,0,0,0,1,0,0
1,DNA,277.0,6,1.90,2337.73,6.6,2.90,57.63,0,0,0,0,0,0,1,0,0
2,DNA,277.0,24,2.25,7374.83,6.6,2.29,46.25,0,0,0,0,0,0,1,0,0
3,DNA,281.0,24,2.50,7356.81,7.4,2.35,47.59,0,0,0,0,0,0,1,0,0
4,DNA,277.0,12,2.55,3663.39,6.5,3.01,59.09,0,0,0,0,0,0,1,0,0


In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82428 entries, 0 to 82427
Data columns (total 17 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   y                                                        82428 non-null  object 
 1   temperatura_cristalizacao_k                              82428 non-null  float64
 2   contagem_residuos                                        82428 non-null  int64  
 3   medida_de_resolucao                                      82428 non-null  float64
 4   peso_molecular                                           82428 non-null  float64
 5   ph                                                       82428 non-null  float64
 6   densidade_matthews                                       82428 non-null  float64
 7   densidade_percentual_solucao                             82428 non-null  float64
 8   tecnica_experimental__ELEC

# Selecionando quais classes o modelo irá receber para prever

In [120]:
df.value_counts('y', normalize=True).reset_index()

,y,proportion
0,HYDROLASE,0.159642
1,TRANSFERASE,0.123283
2,OXIDOREDUCTASE,0.096521
3,LYASE,0.037694
4,IMMUNESYSTEM,0.033241
...,...,...
1603,MEMBRANEPROTEINLIPIDTRANSPORT,0.000012
1604,MEMBRANEPROTEINMETALBINDINGPROTEIN,0.000012
1605,MEMBRANEPROTEINNUCLEARPROTEIN,0.000012
1606,MEMBRANEPROTEINRECEPTOR,0.000012


In [121]:
print("Visualizando a contagem de diferentes classes para Y")

corte = 0.001
df_counts = df.value_counts('y', normalize=True).reset_index().query(f"proportion >= {corte}")

fig = px.bar(df_counts, 
             x='proportion', y='y', 
             title=f"Contagem de classificações que representam, no mínimo, {corte * 100}% de todo o conjunto de dados")
fig.show()

Visualizando a contagem de diferentes classes para Y


Pelo alto volume de classes, apenas algumas classes (as mais frequentes) serão utilizadas como preditoras.

In [164]:
lista_classificadores = df_counts['y'].to_list()[0:4]
df_filter = df.query(f"y in {lista_classificadores}")

print(lista_classificadores)

['HYDROLASE', 'TRANSFERASE', 'OXIDOREDUCTASE', 'LYASE']


# Observando a relação quantidade da variável Target pelas features

In [151]:
fig = px.bar(df_filter.value_counts('y').reset_index(),
             x='count', y='y', 
             title=f"Contagem da variável target")
fig.show()

In [152]:
df_corr = df_filter.drop(columns=['y']).corr()
fig = px.imshow(df_corr, color_continuous_scale='Viridis')

fig.update_layout(title='Mapa de Calor da Correlação')
fig.update_xaxes(title_text='', showticklabels=False)
fig.show()

In [158]:
agg_dict = {}
for col in df_filter.drop(columns=['y']).columns.to_list():
    agg_dict.update({col: 'median'})

df_agg = df_filter.groupby('y').agg(agg_dict).reset_index()

def agg_plot(df, x):
    fig = px.bar(df, x='y', y=x, 
                 title=f'Mediana da {x} para cada classificação',
                 labels={f'{x}': f'Mediana de {x}', 'y': 'Classificação'})
    fig.show()

for feature in df_agg.drop(columns=['y']).columns.to_list():
    agg_plot(df_agg, feature)

Grande maioria das variáveis não apresenta uma grande diferença na mediana, portanto podem não ser as melhroes features para se utilizar no modelo. A baixa variação nos valores não darão grandes insumos para o modelo no momento da previsão.

# Seleção das variáveis por meio de um modelo com base em árvores

Executando a normalização das features numéricas por min max

In [154]:
SEED = 1337

def select_x_features(dataframe, list_x=None, norm=True):
    y = dataframe['y']
    
    if list_x == None:
        x = dataframe.drop(columns=['y'])
    else:
        x = dataframe.drop(columns=['y'])[list_x] 

    if norm==True:
        x = pd.DataFrame(minmax_scale(x, (0, 1)), columns=x.columns.to_list())

    X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=SEED, test_size=0.20)
    model = RandomForestClassifier(random_state=SEED)
    model.fit(X_train, y_train)

    # Exibir performance do modelo
    score_value = model.score(X_test, y_test)
    print(f"Score do modelo {score_value}")

    importances = model.feature_importances_
    feature_importances = [(importance, feature) for importance, feature in zip(importances, x.columns.to_list())]
    feature_importances_sorted = sorted(feature_importances, reverse=True)

    # Exibir as features mais importantes
    for importance, feature_name in feature_importances_sorted:
        print(f'Feature: {feature_name}, Importância: {importance}')

In [155]:
select_x_features(df_filter)

Score do modelo 0.654645921186564
Feature: peso_molecular, Importância: 0.19797553345290184
Feature: contagem_residuos, Importância: 0.1945414042071829
Feature: densidade_percentual_solucao, Importância: 0.1486929996035246
Feature: densidade_matthews, Importância: 0.1261950479442134
Feature: medida_de_resolucao, Importância: 0.12231633241622741
Feature: ph, Importância: 0.12078527351237897
Feature: temperatura_cristalizacao_k, Importância: 0.08927414192594695
Feature: tecnica_experimental__XRAYDIFFRACTION, Importância: 9.609907880411592e-05
Feature: tecnica_experimental__NEUTRONDIFFRACTION, Importância: 5.467143323377609e-05
Feature: tecnica_experimental__SOLUTIONSCATTERINGXRAYDIFFRACTION, Importância: 2.0739462432818023e-05
Feature: tecnica_experimental__NEUTRONDIFFRACTIONXRAYDIFFRACTION, Importância: 1.426195987306137e-05
Feature: tecnica_experimental__ELECTRONCRYSTALLOGRAPHY, Importância: 1.3842899394073911e-05
Feature: tecnica_experimental__POWDERDIFFRACTION, Importância: 1.0985465

Removendo as features de baixissima importância

In [162]:
lista_features = ['temperatura_cristalizacao_k', 'contagem_residuos',
       'medida_de_resolucao', 'peso_molecular', 'ph', 'densidade_matthews',
       'densidade_percentual_solucao']
select_x_features(df_filter, list_x=lista_features)

Score do modelo 0.654645921186564
Feature: peso_molecular, Importância: 0.19922869231807191
Feature: contagem_residuos, Importância: 0.19648575398736817
Feature: densidade_percentual_solucao, Importância: 0.14786615807207298
Feature: densidade_matthews, Importância: 0.12412830544749565
Feature: ph, Importância: 0.12171024203122975
Feature: medida_de_resolucao, Importância: 0.1180109580795071
Feature: temperatura_cristalizacao_k, Importância: 0.09256989006425435


Removendo features com alta correlação entre sí

In [163]:
lista_features = ['temperatura_cristalizacao_k', 'contagem_residuos',
       'medida_de_resolucao', 'peso_molecular', 'ph', 'densidade_percentual_solucao']
select_x_features(df_filter, list_x=lista_features)

Score do modelo 0.6585720517667588
Feature: peso_molecular, Importância: 0.22330179194254526
Feature: contagem_residuos, Importância: 0.22059455192277755
Feature: densidade_percentual_solucao, Importância: 0.19252277244994015
Feature: ph, Importância: 0.133732927302005
Feature: medida_de_resolucao, Importância: 0.1293546068852061
Feature: temperatura_cristalizacao_k, Importância: 0.10049334949752592


# Lista final de targets:
```py:
    ['HYDROLASE', 
    'TRANSFERASE', 
    'OXIDOREDUCTASE', 
    'LYASE']
```

# Lista final de features:
```py:
    ['temperatura_cristalizacao_k',
    'contagem_residuos',
    'medida_de_resolucao', 
    'peso_molecular', 
    'ph', 
    'densidade_percentual_solucao']
```